# Mortgage Rate Model

This is the first model being built using the model building framework

## Packages, Function Imports, and Keys

In [ ]:
from fredapi import Fred
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import clear_output

from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error

from xgboost import XGBRegressor

In [ ]:
fred = Fred(api_key='1e208e2d66ac6382c25f85524a5820cc')

## Data Intake

In [ ]:
def download_fred_data_as_dataframe(series_code = str):
    '''Downloads and cleans FRED API call into dataframe.'''

    # Download the series using fredapi
    fred_series = fred.get_series(series_code)

    # Convert the Series into a DataFrame and move the index into a column
    df_raw = fred_series.reset_index()

    # Convert the series code to lowercase for the column name
    lowercase_series_code = series_code.lower()

    # Explicitly rename columns by position to avoid guessing the column name
    df_raw.columns = ["date", lowercase_series_code]

    # Convert date column to datetime
    df_raw['date'] = pd.to_datetime(df_raw['date'])

    return df_raw

In [ ]:
# Intake mortgage rate time series
df_mortgage_rate_weekly = download_fred_data_as_dataframe('MORTGAGE30US')

print(df_mortgage_rate_weekly)

In [ ]:
# 10 Year Treasury Yield
df_ten_year_yield_daily = download_fred_data_as_dataframe('DGS10')

print(df_ten_year_yield_daily)

In [ ]:
# 2 Year Treasury Yield
df_two_year_yield_daily = download_fred_data_as_dataframe('DGS2')

print(df_two_year_yield_daily)

In [ ]:
# 10Y–2Y spread
df_ten_two_yield_spread_daily = download_fred_data_as_dataframe('T10Y2Y')

print(df_ten_two_yield_spread_daily)

In [ ]:
# Effective Federal funds rate
df_federal_funds_rate_daily = download_fred_data_as_dataframe('DFF')

print(df_federal_funds_rate_daily)

## Clean Data
Follow the formatting of FRED

## Create a dataframe with dates, then merge all data onto it

In [ ]:
df_dates = pd.DataFrame({
    "date": pd.date_range(start="1900-01-01", end=pd.Timestamp.today(), freq="D")
})

## Merge

Merge all data into one big dataframe. We start using a dataframe that contains every date from 1900 to present. We then merge every dataframe containing data (target and explanatory) onto this clean date dataframe. The result is a clean dataframe where we can easily delete each row that does not contain data.

In [ ]:
# List of all dataframes that contain data that will be in our model.
list_data_dfs = [
    df_mortgage_rate_weekly, 
    df_ten_year_yield_daily, 
    df_two_year_yield_daily, 
    df_ten_two_yield_spread_daily, 
    df_federal_funds_rate_daily]

In [ ]:
def merge_dates_with_data(base_dates_dataframe, data_dataframe):
    # Merge a single data dataframe onto the base dates dataframe.
    # This function always returns a NEW dataframe and does not modify inputs.
    # An outer merge is used so no dates are lost.
    merged_dataframe = base_dates_dataframe.merge(
        data_dataframe,
        how="outer",
        on="date"
    )

    return merged_dataframe

In [ ]:
# Start with the master date dataframe
df_all_data = df_dates.copy()

# Merge each dataframe one at a time (explicitly, no loop)
df_all_data = merge_dates_with_data(df_all_data, df_mortgage_rate_weekly)
df_all_data = merge_dates_with_data(df_all_data, df_ten_year_yield_daily)
df_all_data = merge_dates_with_data(df_all_data, df_two_year_yield_daily)
df_all_data = merge_dates_with_data(df_all_data, df_ten_two_yield_spread_daily)
df_all_data = merge_dates_with_data(df_all_data, df_federal_funds_rate_daily)

## Clean the DataFrame

To make this super simple on my self, we will assure datatypes for the whole dataframe, and standardize missing data.

In [ ]:
df_all_data.columns

In [ ]:
# Rename Target column
target_column_name = 'mortgage30us'

df_all_data = df_all_data.rename(columns={target_column_name : 'target'})

In [ ]:
# Drop rows where the target is not present.
df_all_data.dropna(subset=["target"], inplace=True)

In [ ]:
# Assure that numeric cols are numeric and have standardized NaN values
numeric_cols = df_all_data.columns.difference(['date'])
df_all_data[numeric_cols] = (df_all_data[numeric_cols].apply(pd.to_numeric, errors='coerce').replace([np.inf, -np.inf], np.nan))

## Data Engineering and Transforming

12/21/2025 note: VIF is high between most variables. Highest correlation with dgs10. For sake of getting started, I will do a basic model of dgs10 on mortgage rate

In [ ]:
# NOTE: After doing a review of VIF and correlation:
df_model_data = df_all_data.copy()

# df_model_data = df_model_data[['date', 'target', 'dgs10']]

## EDA

### Correlation Matrix

This shows us how closesly correlated two variables are in our data. (> .50 or < -.5 represents higher correlation)

If two variables are correlated, it could represent a possible explanatory relationship.

- The most important variable to have high correlation is with the 'target'.

- If two variables THAT ARE NOT THE 'target' are highly correlated, we could have multicollinearity (bad), so we need to test VIF (below).

> If the correlation is positive, when one variable goes up, so does the other. Same vice versa, if the correlation is negative, when one variable goes up, the other goes down. Think about if this makes sense intuitively.

In [ ]:
corr = df_model_data.drop(columns=["date"]).corr()

sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.show()

### Correlation over time

The matrix above show us the correlation over all of history, but the relationship between variables can change over time. These graphs display how the correlation between the variables and the target changes over time, with a 95% confidence interval. A variable getting close to or crossing 0% could mean that the relationship is not strong, and should probably be removed from the model.

In [ ]:
# --- hardcode these ---
df = df_model_data.copy()
WINDOW = 30
START_DATE = "2009-01-01"   # or None
END_DATE   = None # "2024-12-31"   # or None
# ----------------------

In [ ]:
df = df.sort_values("date").set_index("date")
num = df.select_dtypes(include="number")

feature_cols = [c for c in num.columns if c != "target"]

# rolling correlation (index=date, columns=features)
corr = num[feature_cols].rolling(WINDOW).corr(num["target"])

# rolling sample size (pairwise non-NaN count per window)
n = pd.concat(
    {
        c: num[[c, "target"]].notna().all(axis=1).rolling(WINDOW).sum()
        for c in feature_cols
    },
    axis=1,
)

# hardcoded date filter
if START_DATE is not None:
    start_ts = pd.Timestamp(START_DATE)
    corr = corr.loc[corr.index >= start_ts]
    n = n.loc[corr.index]

if END_DATE is not None:
    end_ts = pd.Timestamp(END_DATE)
    corr = corr.loc[corr.index <= end_ts]
    n = n.loc[corr.index]

plt.style.use("seaborn-v0_8-whitegrid")
ZCRIT = 1.959963984540054  # ~95% CI

def corr_ci(r, n_):
    r = np.asarray(r, float)
    n_ = np.asarray(n_, float)

    lo = np.full_like(r, np.nan)
    hi = np.full_like(r, np.nan)

    ok = np.isfinite(r) & np.isfinite(n_) & (n_ >= 4) & (np.abs(r) < 1)
    if ok.any():
        z = np.arctanh(r[ok])
        se = 1.0 / np.sqrt(n_[ok] - 3.0)
        lo[ok] = np.tanh(z - ZCRIT * se)
        hi[ok] = np.tanh(z + ZCRIT * se)

    return lo, hi

def plot_corr(feature):
    clear_output(wait=True)

    r = corr[feature]
    nn = n[feature]
    lo, hi = corr_ci(r.values, nn.values)

    fig, ax = plt.subplots(figsize=(10, 3.2))
    ax.plot(r.index, r.values, color="#1f77b4", linewidth=2)
    ax.fill_between(r.index, lo, hi, color="#1f77b4", alpha=0.18, linewidth=0)

    ax.axhline(0, color="black", linewidth=1, alpha=0.6)
    ax.set_ylim(-1, 1)
    ax.set_title(f"{feature} vs target — rolling {WINDOW} corr (+ ~95% CI)", fontsize=12)
    ax.set_ylabel("Correlation")
    ax.set_xlabel("")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    plt.tight_layout()
    plt.show()
    plt.close(fig)

widgets.interact(
    plot_corr,
    feature=widgets.Dropdown(options=feature_cols)
)


### Variance Inflation Factor (VIF)

This tests for multicollinearity between columns. Multicollinearity is when two data sources are similar to, or the same, as one another. Multicollinearity is a problem for Linear and Polynomial Regression models. So if the model is only being solved using a basic regression, we should assure little to no multicollinearity.

Care about VIF if:
- You’re doing inference
- You care about coefficient signs/magnitudes
- You’re publishing / explaining results
- You’re using linear or logistic regression

Don’t care much if:
- You only care about prediction
- You’re using trees / boosting / RF
- You use regularization (ridge handles this)

> Values near 1 mean the variable is largely independent; values above ~5–10 indicate strong multicollinearity, and inf means exact redundancy that must be fixed. High VIF affects coefficient stability and interpretability, not predictive accuracy.

In [ ]:
# NOTE: 12/21 - Skipping because we no longer have lots of variables.

import numpy as np
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

df_independents = df_model_data.apply(pd.to_numeric, errors="coerce")
df_independents = df_independents.drop(columns=['date', 'target'])

X = df_independents.dropna().to_numpy()

vif = pd.Series(
    [variance_inflation_factor(X, i) for i in range(X.shape[1])],
    index=df_independents.columns,
    name="VIF",
).sort_values(ascending=False)

vif


### Modeling

In [ ]:
df_model_data

In [ ]:
# How many observations out should be predicted.
PREDICTION_HORIZON = 1
GROWTH_OR_LEVEL = 'growth' #'growth' or 'level'

df_model = df_model_data.copy()

if GROWTH_OR_LEVEL == 'level':
    # Shift the target and date by the prediction horizon
    df_model['target'] = df_model['target'].shift(PREDICTION_HORIZON)
    df_model['target_date'] = df_model['date'].shift(PREDICTION_HORIZON)
    df_model['feature_date'] = df_model['date']
    df_model.drop(columns=['date'])

    # Drop missing data due to the shift
    df_model.dropna(subset=['target'], inplace=True)

else:  # growth
    # Columns to convert to growth
    growth_cols = [c for c in df_model.columns if c not in ['target', 'date']]

    # Convert features to growth
    df_model[growth_cols] = df_model[growth_cols].pct_change(fill_method=None)

    # Convert target to growth separately
    df_model['target'] = df_model['target'].pct_change(fill_method=None)

    # Shift the target and date by the prediction horizon
    df_model['target'] = df_model['target'].shift(PREDICTION_HORIZON)
    df_model['target_date'] = df_model['date'].shift(PREDICTION_HORIZON)
    df_model['feature_date'] = df_model['date']
    df_model.drop(columns=['date'])

    # Drop missing data due to the shift
    df_model.dropna(subset=['target'], inplace=True)


In [ ]:
# NOTE: 12/29/2025 - Skipping for now, because we are trying to do xgboost

# import seaborn as sns
# import matplotlib.pyplot as plt

# target_var = 'target'
# comparison_var = 'dgs10'

# df_plot = df_model[[comparison_var, target_var]].dropna()

# plt.figure()
# sns.regplot(
#     data=df_plot,
#     x=comparison_var,
#     y=target_var,
#     ci=95
# )
# plt.title("target vs dgs10 with OLS fit and 95% CI")
# plt.show()


In [ ]:
# NOTE: 12/29/2025 - Skipping for now, because we are trying to do xgboost

# import statsmodels.formula.api as smf

# model = smf.ols(formula='target ~ dgs10', data=df_model).fit()

# # model summary
# print(model.summary())

In [ ]:
MODEL_NAME = "xgboost"       # "xgboost" or "ridge"
WINDOW_TYPE = "expanding"    # "expanding" or "rolling"
ROLLING_WINDOW_SIZE = 500    # only used if WINDOW_TYPE == "rolling"
MIN_TRAIN_ROWS = 200

df_model["feature_date"] = pd.to_datetime(df_model["feature_date"])
df_model["target_date"] = pd.to_datetime(df_model["target_date"])
df_model = df_model.sort_values("feature_date").reset_index(drop=True)

feature_columns = [c for c in df_model.columns if c not in ["feature_date", "target_date", "target"]]

# Keep it strict: models require numeric features. If you have non-numeric, encode/drop before this.
non_numeric_feature_columns = df_model[feature_columns].select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric_feature_columns:
    raise ValueError(f"Non-numeric feature columns found: {non_numeric_feature_columns}")

X_all_features = df_model[feature_columns]
y_all_target = df_model["target"]

if MODEL_NAME.lower() in {"xgb", "xgboost"}:
    model_template = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        n_jobs=-1,
        objective="reg:squarederror",
        random_state=42,
    )
elif MODEL_NAME.lower() == "ridge":
    model_template = Ridge(alpha=1.0)
else:
    raise ValueError("MODEL_NAME must be 'xgboost' or 'ridge'")

predictions = np.full(len(df_model), np.nan)

for cutoff_feature_date in df_model["feature_date"].drop_duplicates():
    training_rows = df_model.index[df_model["feature_date"] < cutoff_feature_date]
    prediction_rows = df_model.index[df_model["feature_date"] == cutoff_feature_date]

    if len(training_rows) < MIN_TRAIN_ROWS or len(prediction_rows) == 0:
        continue

    if WINDOW_TYPE == "rolling":
        training_rows = training_rows[-ROLLING_WINDOW_SIZE:]
    elif WINDOW_TYPE != "expanding":
        raise ValueError("WINDOW_TYPE must be 'expanding' or 'rolling'")

    model = model_template.__class__(**model_template.get_params())  # fresh copy each step
    model.fit(X_all_features.loc[training_rows], y_all_target.loc[training_rows])
    predictions[prediction_rows] = model.predict(X_all_features.loc[prediction_rows])

backtest_results = df_model[["feature_date", "target_date", "target"]].copy()
backtest_results["prediction"] = predictions
backtest_results = backtest_results.dropna(subset=["prediction"]).reset_index(drop=True)
backtest_results["error"] = backtest_results["prediction"] - backtest_results["target"]

rmse = mean_squared_error(backtest_results["target"], backtest_results["prediction"], squared=False)
mae = mean_absolute_error(backtest_results["target"], backtest_results["prediction"])

print(f"MODEL_NAME={MODEL_NAME}  WINDOW_TYPE={WINDOW_TYPE}")
print(f"RMSE={rmse:.6f}  MAE={mae:.6f}")

# backtest_results has one row per prediction, time-safe
# backtest_results.head()